<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
DataApp - Decision Tree - Diamonds
</b></font> </br></p>

---


In [ ]:
#@title 🔧 Colab-Umgebung { display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from ml_lib.utilities import get_ipinfo
import sys
print()
print(f"Python Version: {sys.version}")
print()
get_ipinfo()

# 0  | Install & Import
***

<p><font color='black' size="5">
Gradio
</font></p>



Gradio ist eine Open-Source-Webplattform, die es Benutzern ermöglicht, benutzerdefinierte Schnittstellen für Machine-Learning-Modelle zu erstellen und zu teilen, ohne dass dafür Webentwicklungsfähigkeiten erforderlich sind. Mit Gradio können Benutzer schnell interaktive Demos erstellen, die Leistung ihres Modells visualisieren und ihre Modelle mit anderen teilen. Die Plattform unterstützt mehrere beliebte Machine-Learning-Bibliotheken, einschließlich TensorFlow, PyTorch und scikit-learn, und bietet eine einfache API für Entwickler, um ihre Modelle in Gradio zu integrieren. Gradio wird häufig für Prototyping, Modell-Debugging und den Bau einfacher ML-Anwendungen verwendet.


[Doku Gradio](https://gradio.app/)

In [ ]:
# Install
!uv pip install --system -q pypmml nyoka

In [ ]:
# Import
import pandas as pd
from pandas import read_csv, DataFrame, to_numeric

from sklearn.preprocessing import OrdinalEncoder

from pypmml import Model
import nyoka
import xml.etree.ElementTree as ET

from gradio import Interface
from gradio.components import Slider, Radio, Dropdown, Number, Textbox

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---


# 2 |  Prepare

---

# 3 |  Modeling
---

# 4 | Evaluate
---


# 5 | Deploy
---

<p><font color='black' size="5">
Modell laden
</font></p>

In [ ]:
# Datei von GitHub kopieren
!curl -L https://raw.githubusercontent.com/ralf-42/ML_Intro/main/04_model/diamonds_model.pmml -o diamonds_model.pmml

In [ ]:
model = Model.fromFile("/content/diamonds_model.pmml")

<p><font color='black' size="5">
Abfragefunktion für DataApp
</font></p>

In [ ]:
cut_seq = ["Fair", "Good", "Very Good", "Premium", "Ideal"]
color_seq = ["J", "I", "H", "G", "F", "E", "D"]
clarity_seq = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]

In [ ]:
def predict_diamonds(*args):
    feature_vector = DataFrame(
        [args]
    )  # Liste in Liste umwandeln, um eine Zeile zu bilden
    feature_vector.columns = [
        "Carat",
        "Cut",
        "Color",
        "Clarity",
        "Depth",
        "Table",
    ]  # Beispielhafte Spaltennamen

    # Konvertiere numerische Spalten
    feature_vector = feature_vector.apply(to_numeric, errors="ignore")

    # Trenne kategoriale und numerische Spalten
    cat_col = feature_vector.select_dtypes(exclude="number").columns

    # OrdinalEncoder korrekt zuweisen
    cat_seq = [cut_seq, color_seq, clarity_seq]
    encoder = OrdinalEncoder(categories=cat_seq)
    feature_vector[cat_col] = encoder.fit_transform(feature_vector[cat_col])

    # Vorbereitung für die Vorhersage
    prepared_features = (
        feature_vector.values
    )  # direkt .values verwenden, um das Array zu erhalten

    # Beispiel Modell-Vorhersage (Modell-Definition hier fiktiv)
    prediction = model.predict(prepared_features)
    return f"{(prediction[0])[0]:,.2f}"

In [ ]:
# Test der Funktion
predict_diamonds(0.7, 'Very Good', 'F', 'SI1', 63.5, 56.0)
# 2,305.27

<p><font color='black' size="5">
App Design
</font></p>

In [ ]:
interface = Interface(
    fn=predict_diamonds,
    inputs=[
        Slider(label="Carat", minimum=0.2, maximum=5.0, step=0.1, value=0.7),
        Radio(label="Cut", choices=cut_seq, value="Very Good"),
        Dropdown(label="Color", choices=color_seq, value="F"),
        Dropdown(label="Clarity", choices=clarity_seq, value="SI1"),
        Slider(
            label="Total Deapth Percentage",
            minimum=43.0,
            maximum=79.0,
            step=0.5,
            value=63.5,
        ),
        Slider(label="Table", minimum=43.0, maximum=95.0, step=0.5, value=56.0),
    ],
    outputs=Textbox(label="Schätzpreis: "),
    title="Schätzpreis für Diamanten",
    description='<p style="text-align: center";>Demo, liefert möglicherweise ungenaue oder falsche Informationen über Preise </br><b>- Alle Angaben ohne Gewähr - </b></p>',
    allow_flagging="never",
)


<p><font color='black' size="5">
Create a local link
</font></p>

In [ ]:
interface.launch()